In [14]:
import boto3

# Initialize the SageMaker client
sagemaker_client = boto3.client('sagemaker')

# List trial components
try:
    # Request to list trial components
    response = sagemaker_client.list_trial_components()

    # Extract trial component summaries
    trial_component_summaries = response['TrialComponentSummaries']
    trial_component_names = [tc['TrialComponentName'] for tc in trial_component_summaries]

    # Print trial component names
    print("Trial Components:")
    for name in trial_component_names:
        print(name)
except Exception as e:
    print(f"An error occurred: {e}")



Trial Components:
sagemaker-xgboost-2024-01-08-22-21-12-629-aws-training-job
sagemaker-xgboost-2024-01-08-20-27-21-957-aws-training-job
sagemaker-xgboost-2024-01-08-20-20-24-130-aws-training-job
test-processing-job-2023-05-09-13-41-50-aws-processing-job
test-processing-job-2023-05-09-12-38-51-aws-processing-job
test-processing-job1-aws-processing-job
test-processing-job-aws-processing-job
pepp-detectron2-test-2022--profilerreport-1668068752-5febdeb9-aws-processing-job
pepp-detectron2-test-2022-11-10-08-25-52-345-aws-training-job
pepp-detectron2-test-2022--profilerreport-1668038719-aae6c7a7-aws-processing-job
pepp-detectron2-test-2022-11-10-00-05-19-631-aws-training-job
dyco3d-2022-01-09-09-50-25-profilerreport-1641721825-576ac66c-aws-processing-job
dyco3d-2022-01-09-09-50-25-407-aws-training-job
pointgroup-2022-01-09-09-3-profilerreport-1641720840-dedeefff-aws-processing-job
pointgroup-2022-01-09-09-34-00-911-aws-training-job
dyco3d-220102-0826-050-9420d20f-aws-training-job
dyco3d-2201

In [15]:
import boto3
import sagemaker

# Initialize SageMaker session and client
sess = sagemaker.Session()
sagemaker_client = boto3.client('sagemaker')

trial_component_name="sagemaker-xgboost-2024-01-08-22-21-12-629-aws-training-job"

# Retrieve trial component information
trial_component = sagemaker_client.describe_trial_component(TrialComponentName=trial_component_name)


In [21]:
trial_component_arn=trial_component["TrialComponentArn"]
hp_artifact = sagemaker_client.list_associations(DestinationArn=trial_component_arn, SourceType='HyperParameters')

In [23]:
hp_artifact.keys()

dict_keys(['AssociationSummaries', 'ResponseMetadata'])

In [26]:
hp_artifact_arn=hp_artifact['AssociationSummaries'][1]['SourceArn']

In [28]:
# Retrieve the artifact's information
try:
    artifact_info = sagemaker_client.describe_artifact(ArtifactArn=hp_artifact_arn)
    print(artifact_info)
except Exception as e:
    print(f"An error occurred: {e}")

{'ArtifactName': 'HP', 'ArtifactArn': 'arn:aws:sagemaker:us-east-1:470086202700:artifact/b10cebaaca6e9b305d367ac5af3774cc', 'Source': {'SourceUri': '{"eta": "0.2", "gamma": "4", "max_depth": "5", "min_child_weight": "6", "num_round": "100", "objective": "binary:logistic", "subsample": "0.8", "verbosity": "0"}', 'SourceTypes': []}, 'ArtifactType': 'HyperParameters', 'Properties': {}, 'CreationTime': datetime.datetime(2024, 1, 9, 10, 1, 23, 786000, tzinfo=tzlocal()), 'CreatedBy': {}, 'LastModifiedTime': datetime.datetime(2024, 1, 9, 10, 1, 23, 786000, tzinfo=tzlocal()), 'LastModifiedBy': {}, 'LineageGroupArn': 'arn:aws:sagemaker:us-east-1:470086202700:lineage-group/sagemaker-default-lineage-group', 'ResponseMetadata': {'RequestId': '9975a509-8e57-4853-b842-6423abf0ee50', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '9975a509-8e57-4853-b842-6423abf0ee50', 'content-type': 'application/x-amz-json-1.1', 'content-length': '615', 'date': 'Tue, 09 Jan 2024 11:24:48 GMT'}, 'RetryAt

In [32]:
# Extract parameters
import json
if 'Source' in artifact_info:
    hp = artifact_info['Source']['SourceUri']
    print(json.loads(hp))
else:
    print("S3 URI not found in the artifact information.")

{'eta': '0.2', 'gamma': '4', 'max_depth': '5', 'min_child_weight': '6', 'num_round': '100', 'objective': 'binary:logistic', 'subsample': '0.8', 'verbosity': '0'}


In [ ]:
# Extracting S3 URIs for input data and script (modify as per your trial component's structure)
train_data_uri = trial_component['InputArtifacts']['train']['Value']
val_data_uri= trial_component['InputArtifacts']['validation']['Value']
training_script_uri = trial_component['Parameters']['SageMaker.ImageUri']['StringValue']

In [ ]:
# Extracting hyperparameters (modify keys according to your trial component's structure)
parameters = trial_component['Parameters']

In [ ]:
parameters

In [ ]:
hyperparameters = {k: float(v['NumberValue']) if 'NumberValue' in v else v['StringValue']
                   for k, v in parameters.items() if "SageMaker" not in k}


In [ ]:
hyperparameters

In [ ]:
sagemaker_iam_role = sagemaker.get_execution_role()

In [ ]:
output_location="s3://sagemaker-us-east-1-470086202700/xgboost-fraud-repr/output"

In [ ]:
xgb = sagemaker.estimator.Estimator(training_script_uri,
                                    role=sagemaker_iam_role, 
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    output_path=output_location,
                                    sagemaker_session=sess)